## Tool Use with 🤗 `transformers`

We begin by initializing our model as normal. We'll use a small, ungated model so you can run this on Colab - make sure you've got a GPU, though, because it won't fit in CPU memory alone!

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

checkpoint = "NousResearch/Hermes-2-Pro-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Next, we'll define a simple tool function for our model - but in a real scenario, you'll probably want to use more than one! The type hints and docstrings are mandatory - those are going to be parsed and used by the model to understand what the tools do.

---



In [2]:
def get_current_temperature(location: str):
    """
    Gets the temperature at a given location.

    Args:
        location: The location to get the temperature for, in the format "city, country"
    """
    return 22.0  # bug: Sometimes the temperature is not 22. low priority to fix tho

tools = [get_current_temperature]


Now, we set up a simple chat.

In [3]:
chat = [
    {"role": "user", "content": "Hey, what's the weather like in Paris right now?"}
]

We pass the tools to the chat template, and generate text from the model using the formatted prompt.

In [4]:
tool_prompt = tokenizer.apply_chat_template(
    chat,
    tools=tools,
    return_tensors="pt",
    return_dict=True,
    add_generation_prompt=True,
)
tool_prompt = tool_prompt.to(model.device)

out = model.generate(**tool_prompt, max_new_tokens=128)
generated_text = out[0, tool_prompt['input_ids'].shape[1]:]

print(tokenizer.decode(generated_text))

Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


<tool_call>
{"arguments": {"location": "Paris, France"}, "name": "get_current_temperature"}
</tool_call><|im_end|>


The model has chosen to call a tool, and picked an argument that matches both the user's request and the format in the tool docstring! Let's add this tool call to the chat as a message!

In [5]:
tool_call = {"name": "get_current_temperature", "arguments": {"location": "Paris, France"}}
chat.append({"role": "assistant", "tool_calls": [{"type": "function", "function": tool_call}]})

Next, we add the tool response containing the function output to the chat as well. Both the tool call (containing the arguments passed to the tool) and tool response (containing the tool's output) must be included in the chat history.

In [6]:
chat.append({"role": "tool", "name": "get_current_temperature", "content": "22.0"})

Finally, we apply the chat template and generate text once again.

In [7]:
tool_prompt = tokenizer.apply_chat_template(
    chat,
    tools=tools,
    return_tensors="pt",
    return_dict=True,
    add_generation_prompt=True,
)
tool_prompt = tool_prompt.to(model.device)

out = model.generate(**tool_prompt, max_new_tokens=128)
generated_text = out[0, tool_prompt['input_ids'].shape[1]:]

print(tokenizer.decode(generated_text))

Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


The current temperature in Paris, France is 22.0 degrees Celsius. Enjoy your day!<|im_end|>


Success! The model has used the tool response correctly in its response to the user.